# Importing Libraries

In [5]:
from bs4 import BeautifulSoup
import requests # library to handle requests
source=requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(source,"lxml")

In [6]:
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab

import folium # map rendering library

In [7]:
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import pandas as pd
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import numpy as np

# Scraping the HTML to a dataframe

In [8]:
Borough = {}
Neighborhood = {}

for location_row in soup.table.find_all('tr'):
    location_data=location_row.text
    postcode=location_data.split('\n')[1]
    borough=location_data.split('\n')[2]
    neighborhood=location_data.split('\n')[3]
    if(borough != "Not assigned"):
        Borough[postcode] = borough
        if(neighborhood != "Not assigned"):
            if(postcode in Neighborhood.keys()):
                Neighborhood[postcode] = Neighborhood[postcode] + ", " + neighborhood
            else:
                Neighborhood[postcode] = neighborhood
        else:
            neighborhood = borough
            if(postcode in Neighborhood.keys()):
                Neighborhood[postcode] = Neighborhood[postcode] + ", " + neighborhood
            else:
                Neighborhood[postcode] = neighborhood


location_df = pd.DataFrame(index=np.arange(len(Borough.keys())),columns=['PostalCode', 'Borough', 'Neighborhood'])

          

In [9]:
i=0
for postcode in Borough.keys():
    location_df.loc[i,'PostalCode']=postcode
    location_df.loc[i,'Borough']=Borough[postcode]
    location_df.loc[i,'Neighborhood']=Neighborhood[postcode]
    i=i+1


# Dataframe for the first part: Dataframe with Postal Code and Borough and Neighborhood

In [10]:
location_df = location_df.drop(0)
location_df

,PostalCode,Borough,Neighborhood
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,"Harbourfront, Regent Park"
4,M6A,North York,"Lawrence Heights, Lawrence Manor"
5,M7A,Queen's Park,Queen's Park
6,M9A,Etobicoke,Islington Avenue
7,M1B,Scarborough,"Rouge, Malvern"
8,M3B,North York,Don Mills North
9,M4B,East York,"Woodbine Gardens, Parkview Hill"
10,M5B,Downtown Toronto,"Ryerson, Garden District"



# Importing the Latitude and Longitudes CSV file

In [11]:
coordinate_df = pd.read_csv("Geospatial_Coordinates.csv")
coordinate_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [20]:
location_with_coordinate_df = pd.DataFrame(index=range(1,len(Borough.keys())),columns=['PostalCode', 'Borough', 'Neighborhood','Latitude','Longitude'])

In [21]:
location_with_coordinate_df.loc[location_df.index.tolist()[0],'PostalCode'] = location_df.loc[location_df.index.tolist()[0],'PostalCode']   

In [22]:
coordinate_df.index = coordinate_df['Postal Code']
coordinate_df.head()

,Postal Code,Latitude,Longitude
Postal Code,,,
M1B,M1B,43.806686,-79.194353
M1C,M1C,43.784535,-79.160497
M1E,M1E,43.763573,-79.188711
M1G,M1G,43.770992,-79.216917
M1H,M1H,43.773136,-79.239476


In [23]:
for row in range(1,len(Borough.keys())):   
    location_with_coordinate_df.loc[row,'PostalCode'] = location_df.loc[row,'PostalCode']
    location_with_coordinate_df.loc[row,'Borough'] = location_df.loc[row,'Borough']
    location_with_coordinate_df.loc[row,'Neighborhood'] = location_df.loc[row,'Neighborhood']
    location_with_coordinate_df.loc[row,'Latitude'] = coordinate_df.loc[location_df.loc[row,'PostalCode'],'Latitude'] 
    location_with_coordinate_df.loc[row,'Longitude'] = coordinate_df.loc[location_df.loc[row,'PostalCode'],'Longitude'] 

In [25]:
location_with_coordinate_df


,PostalCode,Borough,Neighborhood,Latitude,Longitude
1,M3A,North York,Parkwoods,43.7533,-79.3297
2,M4A,North York,Victoria Village,43.7259,-79.3156
3,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.6543,-79.3606
4,M6A,North York,"Lawrence Heights, Lawrence Manor",43.7185,-79.4648
5,M7A,Queen's Park,Queen's Park,43.6623,-79.3895
6,M9A,Etobicoke,Islington Avenue,43.6679,-79.5322
7,M1B,Scarborough,"Rouge, Malvern",43.8067,-79.1944
8,M3B,North York,Don Mills North,43.7459,-79.3522
9,M4B,East York,"Woodbine Gardens, Parkview Hill",43.7064,-79.3099
10,M5B,Downtown Toronto,"Ryerson, Garden District",43.6572,-79.3789


# Finding Location Coordinates for Toronto

In [26]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


# create map of Toronto using latitude and longitude values

In [28]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

In [32]:
# add markers to map

for lat, lng, borough, neighborhood in zip(location_with_coordinate_df['Latitude'], location_with_coordinate_df['Longitude'], location_with_coordinate_df['Borough'], location_with_coordinate_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
map_toronto.save("map_toronto.png")
map_toronto